In [7]:
#import labels
from pyseq import image_analysis as ia
import squidpy as sq
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as dd
import dask.array as da
import dask.bag as db
import skimage
import pandas as pd
import anndata as ad
from shapely.geometry import Point
import geopandas
from libpysal import weights, examples
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

labels = skimage.io.imread('/gpfs/commons/home/jsingh/outputs/masks_final.tiff')
im = ia.get_HiSeqImages(image_path = '/gpfs/commons/home/jsingh/zarrs/m387ntga2.zarr')
one_z_plane = im.im.sel(obj_step = 8498, channel = 558, cycle=1)
arr = one_z_plane.values

/gpfs/commons/home/jsingh/.conda/envs/spatial/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1dev-CAPI-1.14.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/gpfs/commons/home/jsingh/PySeq2500/pyseq/image_analysis.py:955: RuntimeWarning: Failed to open Zarr store with consolidated metadata, falling back to try reading non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  im = xr.open_zarr(fn).to_array()


ImageAnalysis::Opened m387ntga2 


In [8]:
img = skimage.io.imread('/gpfs/commons/home/jsingh/outputs/masks_final.tiff')
ski_img = skimage.measure.regionprops(img)

In [10]:
adata = ad.read_h5ad('/gpfs/commons/groups/nygcfaculty/PySeq/spatial_analysis/m387ntga2_labels_anndata_sample.h5ad')

/gpfs/commons/home/jsingh/.conda/envs/spatial/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [ ]:
# Centroid Distance Matrix
x_coord = []
y_coord = []
for x in range(np.shape(adata.uns['centroid'])[0]):
    y_coord.append(adata.uns['centroid'][x][0])
    x_coord.append(adata.uns['centroid'][x][1])
    
mat = np.column_stack((x_coord, y_coord))

point_list = []
for s in mat:
    point_list.append(Point(s))
    
d = {'geometry': point_list}
gdf = geopandas.GeoDataFrame(d,crs="EPSG:4326")

def graph(n):
    knn3 = weights.KNN.from_dataframe(gdf, k=n) #for k = 3
    knn_graph = knn3.to_networkx()
    return knn_graph